In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
import time

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
# from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import precision_recall_curve, auc, accuracy_score, confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns # plotting problem

!pip install contractions
import contractions # expanding contractions in text can't -> cannot

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 10.6 MB/s eta 0:00:00
/kaggle/input/amazon-product-review-spam-and-non-spam/Home_and_Kitchen/Home_and_Kitchen.json
/kaggle/input/amazon-product-review-spam-and-non-spam/part.json/part.json
/kaggle/input/amazon-product-review-spam-and-non-spam/Electronics/Electronics.json
/kaggle/input/amazon-product-review-spam-and-non-spam/separate.json/separate.json
/kaggle/input/amazon-product-review-spam-and-non-spam/Clothing_Shoes_and_Jewelry/Clothing_Shoes_and_Jewelry.json
/kaggle/input/amazon-product-review-spam-and-non-spam/Sports_and_Outdoors/Sports_and_Outdoors.json
/kaggle/input/amazon-product-review-spam-and-non-spam/Cell_Phones_and_Accessories/Cell_Phones_and_Accessories.json
/kaggle/input/amazon-product-review-spam-and-non-spam/Toys_and_Games/Toys_and_Games.json


In [2]:
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 32.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [3]:
torch.backends.cudnn.deterministic = True
random.seed(21)
np.random.seed(21)
torch.manual_seed(21)
torch.cuda.manual_seed_all(21)

In [4]:
import wandb
wandb.login(key = '02c8923278a3dc82932fafb9959cd6d7587dacc7')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [5]:
df = pd.read_json('/kaggle/input/amazon-product-review-spam-and-non-spam/Toys_and_Games/Toys_and_Games.json', lines=True).head(100000)
df = df.loc[:, ['reviewText', 'class']]

In [6]:
df['reviewText'] = df['reviewText'].apply(lambda x: contractions.fix(x))
df.loc[:, 'reviewText'] = df['reviewText'].str.lower()

# \W represents Special characters like "$" and "!!!"
df.loc[:, 'reviewText'] = df['reviewText'].str.replace('\W', ' ')

# \d represents Numeric digits like "19.99"
df.loc[:, 'reviewText'] = df['reviewText'].str.replace('\d', ' ')

df

,reviewText,class
0,i love these felt nursery rhyme characters and...,1
1,i see no directions for its use. therefore i h...,0
2,this is a great tool for any teacher using the...,1
3,"great product, thank you! our son loved the pu...",1
4,although not as streamlined as the algebra i m...,1
...,...,...
99995,received this product in a timely fashion. i m...,0
99996,mcfarlane sports series are fantastic and life...,1
99997,fortune is a good figure. she has a very attra...,1
99998,i just thought that i would jot a few words to...,0


In [7]:
def pr_auc_score(y_test, y_pred):
    precision, recall, _ = precision_recall_curve(y_test, y_pred)
    pr_auc = auc(recall, precision)
    return pr_auc

In [8]:
y = df['class']
X_train, X_test, y_train, y_test = train_test_split(df['reviewText'], y, test_size=0.33, random_state=21)

In [9]:
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [10]:
# class LSTMClassifierSpaCy(nn.Module):
#     def __init__(self, input_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout):
#         super(LSTMClassifierSpaCy, self).__init__()

#         self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional, batch_first=True, dropout=dropout)
#         self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
#         self.dropout = nn.Dropout(dropout)

#     def forward(self, input_ids):
#         _, (hidden, _) = self.lstm(input_ids)

#         if self.lstm.bidirectional:
#             hidden = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
#         else:
#             hidden = hidden[-1, :, :]

#         hidden = self.dropout(hidden)
#         return self.fc(hidden)

In [11]:
class GRUClassifierSpaCy(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout):
        super(GRUClassifierSpaCy, self).__init__()
        self.gru = nn.GRU(input_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional, batch_first=True, dropout=dropout if n_layers > 1 else 0)
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_ids):
        _, hidden = self.gru(input_ids)

        if self.gru.bidirectional:
            hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        else:
            hidden = hidden[-1,:,:]

        hidden = self.dropout(hidden)
        return self.fc(hidden)

In [12]:
# Training loop
def train_model(model, data_loader, optimizer, criterion, device, epoch):
    model.train()
    for batch in data_loader:
        input_ids = batch['input_ids'].to(device)
        #attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}], Loss: {loss.item():.4f}')
    wandb.log({"epoch": epoch,"loss": loss})

In [13]:
# Evaluation loop
def eval_model(model, data_loader, device):
    model.eval()
    predictions = []
    labels = []

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            #attention_mask = batch['attention_mask'].to(device)
            label = batch['label'].to(device)

            outputs = model(input_ids=input_ids)
            _, pred = torch.max(outputs, dim=1)

            predictions.extend(pred.cpu().numpy())
            labels.extend(label.cpu().numpy())

    return accuracy_score(labels, predictions), pr_auc_score(labels, predictions), classification_report(labels, predictions)

In [14]:
def train_eval(model, train_data_loader, test_data_loader, criterion, optimizer, EPOCHS, device):
    wandb.watch(model, criterion, log="all", log_freq=10)

    start_time = time.time()
    # Train and evaluate the model
    EPOCHS = EPOCHS

    for epoch in range(EPOCHS):
        train_model(model, train_data_loader, optimizer, criterion, device, epoch)

    accuracy, pr_auc, report = eval_model(model, test_data_loader, device)

    wandb.log({"test_accuracy": accuracy})
    wandb.log({"test_pr_auc": pr_auc})

    print(f"Accuracy: {accuracy:.4f}")
    print(f"PR_AUC: {pr_auc:.4f}")
    print(f"Classification Report:\n{report}")

    elapsed_time = time.time() - start_time
    wandb.log({"run_time_sec": elapsed_time})
    wandb.finish()

In [15]:
import spacy
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

s_time = time.time()
# Load SpaCy English model
nlp = spacy.load('en_core_web_md')  # You may have to download this model

# Example: SpaCy embeddings extraction function
def spacy_embedding(text):
    doc = nlp(text)
    return [token.vector for token in doc]

# Dataset class using SpaCy embeddings
class SpacyTextDataset(Dataset):
    def __init__(self, texts, labels, max_length=128):
        self.texts = texts
        self.labels = labels
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = int(self.labels[idx])

        # Retrieve embeddings
        embedding = spacy_embedding(text)
        embedding = embedding[:self.max_length]  # Trim to max_length if too long
        padding = [[0] * 300] * (self.max_length - len(embedding))
        embedding = torch.tensor(embedding + padding, dtype=torch.float)

        return {
            'input_ids': embedding,
            'label': torch.tensor(label, dtype=torch.long)
        }

# Create data loader using SpaCy embedding dataset
def create_spacy_data_loader(X, y, max_length, batch_size):
    dataset = SpacyTextDataset(X, y, max_length)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Example usage:
MAX_LENGTH = 128
BATCH_SIZE = 64
train_data_loader = create_spacy_data_loader(X_train, y_train, MAX_LENGTH, BATCH_SIZE)
test_data_loader = create_spacy_data_loader(X_test, y_test, MAX_LENGTH, BATCH_SIZE)
e_time = time.time() - s_time
print(f"Word Embedding by Spacy ready time: {e_time:.4f}")

# wandb.init(project='RNN Spam Detection', name='LSTM Spacy WEmbedding')

# model = LSTMClassifierSpaCy(input_dim=300, hidden_dim=256, output_dim=2, n_layers=2, bidirectional=True, dropout=0.3)
# model = torch.nn.DataParallel(model)
# model = model.to(device)

# # Set the optimizer and loss function
# optimizer = optim.Adam(model.parameters(), lr=1e-4)
# criterion = nn.CrossEntropyLoss().to(device)

# EPOCHS = 10
# train_eval(model, train_data_loader, test_data_loader, criterion, optimizer, EPOCHS, device)

Word Embedding by Spacy ready time: 2.3102


In [16]:
wandb.init(project='RNN Spam Detection', name='GRU Spacy WEmbedding')

model = GRUClassifierSpaCy(input_dim=300, hidden_dim=256, output_dim=2, n_layers=2, bidirectional=True, dropout=0.3)
model = torch.nn.DataParallel(model)
model = model.to(device)

# Set the optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss().to(device)

EPOCHS = 10
train_eval(model, train_data_loader, test_data_loader, criterion, optimizer, EPOCHS, device)

wandb: Currently logged in as: mint21. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.17.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.16.6
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240512_095626-hs5kloy5
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run GRU Spacy WEmbedding
wandb: ⭐️ View project at https://wandb.ai/mint21/RNN%20Spam%20Detection
wandb: 🚀 View run at https://wandb.ai/mint21/RNN%20Spam%20Detection/runs/hs5kloy5
/tmp/ipykernel_24/2189551758.py:33: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:261.)
  embedding = torch.tensor(embedding + padding, dtype=torch.float)


Epoch [1], Loss: 0.2597
Epoch [2], Loss: 0.1431
Epoch [3], Loss: 0.2031
Epoch [4], Loss: 0.1532
Epoch [5], Loss: 0.1566
Epoch [6], Loss: 0.0907
Epoch [7], Loss: 0.1440
Epoch [8], Loss: 0.1151
Epoch [9], Loss: 0.0426
Epoch [10], Loss: 0.1084
Accuracy: 0.9147
PR_AUC: 0.9701
Classification Report:
              precision    recall  f1-score   support

           0       0.69      0.65      0.67      4420
           1       0.95      0.96      0.95     28580

    accuracy                           0.91     33000
   macro avg       0.82      0.80      0.81     33000
weighted avg       0.91      0.91      0.91     33000



wandb:                                                                                
wandb: 
wandb: Run history:
wandb:         epoch ▁▂▃▃▄▅▆▆▇█
wandb:          loss █▄▆▅▅▃▄▃▁▃
wandb:  run_time_sec ▁
wandb: test_accuracy ▁
wandb:   test_pr_auc ▁
wandb: 
wandb: Run summary:
wandb:         epoch 9
wandb:          loss 0.10841
wandb:  run_time_sec 20085.9077
wandb: test_accuracy 0.91467
wandb:   test_pr_auc 0.97005
wandb: 
wandb: 🚀 View run GRU Spacy WEmbedding at: https://wandb.ai/mint21/RNN%20Spam%20Detection/runs/hs5kloy5
wandb: ⭐️ View project at: https://wandb.ai/mint21/RNN%20Spam%20Detection
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20240512_095626-hs5kloy5/logs
